In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json

Brief intro into the bpps files (base pairing probability matrix of every possible base pair). These are 107 X 107 symmetric matrices, 107 is the number of bases in a RNA sequence

In [8]:
data = np.load('/kaggle/input/stanford-covid-vaccine/bpps/id_001f94081.npy')
print(data.shape)
print(data)

(107, 107)
[[0.         0.         0.         ... 0.         0.         0.0146736 ]
 [0.         0.         0.         ... 0.         0.         0.00620374]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.0146736  0.00620374 0.         ... 0.         0.         0.        ]]


A detailed look at the bpps data

In [13]:
pd.set_option('display.max_colwidth', None)
bpps_data_list = list(data)
bpps_data_list_68 = bpps_data_list[0:68]
bpps_data_list_68_68 = []

for i in range(len(bpps_data_list_68)):
    bpps_data_list_68_68.append(bpps_data_list_68[i][0:68])

print(bpps_data_list_68_68[0])
print(bpps_data_list_68_68[1])

[0.         0.         0.         0.         0.         0.
 0.         0.00655413 0.00922102 0.00809437 0.0217857  0.
 0.         0.         0.         0.         0.00112002 0.
 0.         0.         0.         0.         0.         0.00465301
 0.00241507 0.         0.         0.         0.         0.0114696
 0.00492862 0.         0.00353196 0.         0.         0.
 0.         0.         0.         0.00159406 0.00629747 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.0052959  0.00605378 0.
 0.         0.         0.0012753  0.0136668  0.         0.
 0.00152313 0.        ]
[0.         0.         0.         0.         0.         0.
 0.         0.0182053  0.00511209 0.0386527  0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.0043234
 0.         0.         0.         0.         0.         0.00856126
 0

Ingesting the training dataset which is in json format, then bringing it in pandas dataframe

In [14]:
pd.set_option('display.max_colwidth', None)
json_data = [json.loads(line) for line in open('/kaggle/input/stanford-covid-vaccine/train.json', 'r')]
train_df = pd.DataFrame(json_data)
print(train_df.shape)
train_df.head(2).T

(2400, 19)


,0,1
index,0,1
id,id_001f94081,id_0049f53ba
sequence,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACCGCGCGCGCUUAUGCAAGUUGCCCGCGGCGUUCGCGCUGUGAAAAGAAACAACAACAACAAC
structure,.....((((((.......)))).)).((.....((..((((((....))))))..)).....))....(((((((....))))))).....................,.....(((((((((((((((((((((((....)))))))))).)))))))))))))..(((...))).(((((((....))))))).....................
predicted_loop_type,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHHHSSSSSSIISSIIIIISSXXXXSSSSSSSHHHHSSSSSSSEEEEEEEEEEEEEEEEEEEEE,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSSSSSSSSSSSSMMSSSHHHSSSMSSSSSSSHHHHSSSSSSSEEEEEEEEEEEEEEEEEEEEE
signal_to_noise,6.894,0.193
SN_filter,1.0,0.0
seq_length,107,107
seq_scored,68,68
reactivity_error,"[0.1359, 0.207, 0.1633, 0.1452, 0.1314, 0.105, 0.0821, 0.0964, 0.0756, 0.1087, 0.1377, 0.1544, 0.1622, 0.1388, 0.1284, 0.1009, 0.0941, 0.0564, 0.0417, 0.0596, 0.0482, 0.1041, 0.0942, 0.052, 0.0583, 0.0403, 0.0491, 0.1003, 0.0525, 0.081, 0.1103, 0.0707, 0.0797, 0.0997, 0.0968, 0.0939, 0.0931, 0.0604, 0.0427, 0.0331, 0.0412, 0.0286, 0.0415, 0.0394, 0.0636, 0.0816, 0.0474, 0.0295, 0.0163, 0.0211, 0.0426, 0.0592, 0.0778, 0.0579, 0.0606, 0.0462, 0.0354, 0.0324, 0.0351, 0.0539, 0.0565, 0.0489, 0.0221, 0.0254, 0.0328, 0.0394, 0.0409, 0.0547]","[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 4.2902, 2.3644, 2.3644, 2.3644, 2.3644, 2.3644, 2.3644, 3.0009, 2.3356, 2.923, 4.4649, 4.0411, 2.6232, 2.4985, 2.3879, 1.467, 1.467, 1.467, 1.467, 1.467, 1.467, 2.2892, 1.4271, 2.2008, 1.3919, 1.3919, 1.3919, 2.1211, 2.1008, 2.028, 1.3001, 1.3001, 1.3001, 1.962, 1.9018, 1.8467, 1.2307, 2.0146, 1.7072, 1.6677, 1.165, 2.0515, 1.7922, 1.5873, 1.5375, 1.4931, 1.3286, 2.273, 0.976, 0.976]"


Bringing the bpps data of every sample

In [16]:
list_of_ids = train_df['id'].tolist()
new_train_df = pd.DataFrame()

for i in range(len(list_of_ids)):
#for i in range(5):
    #print(list_of_ids[i])
    file_location = "".join(['/kaggle/input/stanford-covid-vaccine/bpps/',list_of_ids[i],'.npy'])
    bpps_data = np.load(file_location)
    bpps_data_list = list(bpps_data)
    bpps_data_list_68 = bpps_data_list[0:68]
    bpps_data_list_68_68 = []
    for j in range(len(bpps_data_list_68)):
        bpps_data_list_68_68.append(bpps_data_list_68[j][0:68])
    
    new_train_df_single = pd.DataFrame()
    new_train_df_single = train_df.loc[train_df['id']==list_of_ids[i]]
    #print(train_df[train_df['id']==list_of_ids[i]][['id','sequence']].T)

    new_train_df_single['bpps'] = pd.Series([bpps_data_list_68_68], index=new_train_df_single.index)
    #print(new_train_df_single[['id','sequence']].head(2))
    if i == 0:
        new_train_df = new_train_df_single
        #print(new_train_df[['id','signal_to_noise']].head(2))
    else:
        new_train_df = pd.concat([new_train_df, new_train_df_single])
        #print(new_train_df[['id','signal_to_noise']].head(2))
        
new_train_df.shape

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(2400, 20)

Checking whether the new dataframe is consistent with 68 data points for every sample (since we are considering only the first 68 out of the 107 bases)

In [17]:
print(len(new_train_df['deg_50C']))
print(len(new_train_df['deg_error_50C'][4]))
print(len(new_train_df['bpps']))
print(len(new_train_df['bpps'][2399]))
print(len(new_train_df['deg_error_Mg_50C'][1]))

2400
68
2400
68
68


In [89]:
new_train_df[['id','signal_to_noise','bpps']].head(2).T

0  \
id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         